In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import numpy as np
import random
import sys
sys.path.append('tools')

import os
import argparse
from dataset import get_dataset, get_handler, get_wa_handler
from torchvision import transforms
import torch
import csv
import time

import query_strategies
import models
from utils import print_log
import kaggle_data_utility
import dataset
import base_model
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks

In [3]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F

from lightly.data import LightlyDataset, SwaVCollateFunction
from lightly.loss import SwaVLoss
from lightly.loss.memory_bank import MemoryBankModule
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes


class SwaV(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SwaVProjectionHead(512, 512, 128)
        self.prototypes = SwaVPrototypes(128, 512, 5)

        self.start_queue_at_epoch = 30
        self.queues = nn.ModuleList([MemoryBankModule(size=512) for _ in range(2)])

    def forward(self, high_resolution, low_resolution, epoch):
        self.prototypes.normalize()

        high_resolution_features = [self._subforward(x) for x in high_resolution]
        low_resolution_features = [self._subforward(x) for x in low_resolution]

        high_resolution_prototypes = [
            self.prototypes(x, epoch) for x in high_resolution_features
        ]
        low_resolution_prototypes = [
            self.prototypes(x, epoch) for x in low_resolution_features
        ]
        queue_prototypes = self._get_queue_prototypes(high_resolution_features, epoch)

        return high_resolution_prototypes, low_resolution_prototypes, queue_prototypes

    def _subforward(self, input):
        features = self.backbone(input).flatten(start_dim=1)
        features = self.projection_head(features)
        features = nn.functional.normalize(features, dim=1, p=2)
        return features

    @torch.no_grad()
    def _get_queue_prototypes(self, high_resolution_features, epoch):
        if len(high_resolution_features) != len(self.queues):
            raise ValueError(
                f"The number of queues ({len(self.queues)}) should be equal to the number of high "
                f"resolution inputs ({len(high_resolution_features)}). Set `n_queues` accordingly."
            )

        # Get the queue features
        queue_features = []
        for i in range(len(self.queues)):
            _, features = self.queues[i](high_resolution_features[i], update=True)
            # Queue features are in (num_ftrs X queue_length) shape, while the high res
            # features are in (batch_size X num_ftrs). Swap the axes for interoperability.
            features = torch.permute(features, (1, 0))
            queue_features.append(features)

        # If loss calculation with queue prototypes starts at a later epoch,
        # just queue the features and return None instead of queue prototypes.
        if self.start_queue_at_epoch > 0 and epoch < self.start_queue_at_epoch:
            return None

        # Assign prototypes
        queue_prototypes = [self.prototypes(x, epoch) for x in queue_features]
        return queue_prototypes

In [4]:
import torchvision.transforms as T
from torch.utils.data import Dataset
class ActiveDataHandler(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform['transform']
        self.mean = transform['mean']
        self.sd = transform['std']

        self.augmentation = T.Compose([
            T.Normalize([self.mean, self.mean, self.mean], [self.sd, self.sd, self.sd]),
            T.ToPILImage(),
            T.Resize((256, 512), interpolation=T.InterpolationMode.NEAREST),
            T.RandomApply([T.ColorJitter()], p=0.5),
            T.RandomApply([T.GaussianBlur(kernel_size=(3, 3))], p=0.5),
            T.RandomInvert(p=0.2),
            T.RandomPosterize(4, p=0.2),
            T.ToTensor()
        ])

        self.totensor = T.Compose([
            T.Normalize([self.mean, self.mean, self.mean], [self.sd, self.sd, self.sd]),
            T.ToPILImage(),
            T.Resize((256, 512), interpolation=T.InterpolationMode.NEAREST),
            T.ToTensor()
        ])

    def __getitem__(self, index):
        x = self.X[index]
        y = self.Y[index]

        if self.transform:
            channel_index = np.arange(x.shape[0])
            np.random.shuffle(channel_index)
            x = x[channel_index]
            x = torch.from_numpy(x).clone()
            x = x.repeat(3, 1, 1)
            x = self.augmentation(x)

        else:
            x = torch.from_numpy(x).clone()
            x = x.repeat(3, 1, 1)
            x = self.totensor(x)

        return x, torch.from_numpy(y).to(dtype=torch.long), index

    def __len__(self):
        return len(self.X)

In [5]:
data_dir = "data/competition_data/clips"

targets = [
    'Dog_1',
    'Dog_2',
    'Dog_3',
    'Dog_4',
]

# targets = [
#     'Patient_1',
#     'Patient_2',
#     'Patient_3',
#     'Patient_4',
#     'Patient_5',
#     'Patient_6',
#     'Patient_7',
#     'Patient_8'
# ]

ictal_data_list = [kaggle_data_utility.parse_input_data(data_dir, targets[i], 'ictal', None) for i in
                   range(len(targets))]
interictal_data_list = [kaggle_data_utility.parse_input_data(data_dir, targets[i], 'interictal', None) for i in
                        range(len(targets))]
ictal_data_X = np.empty((0, 16, 400))
interictal_data_X = np.empty((0, 16, 400))
for data in ictal_data_list:
    ictal_data_X = np.vstack((ictal_data_X, data['X']))
for data in interictal_data_list:
    interictal_data_X = np.vstack((interictal_data_X, data['X']))
ictal_data_y = np.ones(len(ictal_data_X))[:, np.newaxis]
interictal_data_y = np.zeros(len(interictal_data_X))[:, np.newaxis]
data = np.vstack((ictal_data_X, interictal_data_X))
label = np.vstack((ictal_data_y, interictal_data_y))

Loading data
(0s)
X (178, 16, 400) y (178,) latencies (178,)
Loading data
(0s)
X (172, 16, 400) y (172,) latencies (172,)
Loading data
(2s)
X (480, 16, 400) y (480,) latencies (480,)
Loading data
(1s)
X (257, 16, 400) y (257,) latencies (257,)
Loading data
(1s)
X (418, 16, 400) y (418,)
Loading data
(5s)
X (1148, 16, 400) y (1148,)
Loading data
(22s)
X (4760, 16, 400) y (4760,)
Loading data
(12s)
X (2790, 16, 400) y (2790,)


In [6]:
data.shape
label.shape

(10203, 1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.50, random_state=42)

In [8]:
nStart = 3
nEnd = 30
nQuery = 2

n_pool = len(y_train)
n_test = len(y_test)

In [9]:
NUM_INIT_LB = int(nStart * n_pool / 100)
NUM_QUERY = int(nQuery * n_pool / 100) if nStart != 100 else 0
NUM_ROUND = int((int(nEnd * n_pool / 100) - NUM_INIT_LB) / NUM_QUERY) if nStart != 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd * n_pool / 100) - NUM_INIT_LB) % NUM_QUERY != 0:
        NUM_ROUND += 1

print(NUM_INIT_LB)
print(NUM_QUERY)
print(NUM_ROUND)

153
102
14


In [10]:
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_tmp = np.arange(n_pool)
np.random.shuffle(idxs_tmp)
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True

In [11]:
ckpt = torch.load("ckpt/checkpoint99.pth")
resnet = torchvision.models.resnet34()
backbone = nn.Sequential(*list(resnet.children())[:-1])
swav = SwaV(backbone)
swav.load_state_dict(ckpt['model_state_dict'])
model = base_model.ActiveLearning(swav.backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

idxs_train = np.arange(n_pool)[idxs_lb]

checkpoint_callback = pl_callbacks.ModelCheckpoint(monitor='val_acc', filename='active_ent_round_0-{epoch:02d}-{val_loss:.5f}', dirpath='active_checkpoints_ent')
csv_logger = pl_loggers.CSVLogger("active_logs_ent", name="logger_round_0")
trainer = pl.Trainer( logger=csv_logger, max_epochs=50, callbacks=[checkpoint_callback],accelerator='gpu', devices=1,log_every_n_steps=10)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
from copy import deepcopy
modelstate = deepcopy(model.state_dict())

In [13]:
def collate_fn(batch):
    info = list(zip(*batch))
    data = info[0]
    label = info[1]


    return torch.stack(data), torch.stack(label)
transforms_param = {'transform_tr': {'transform': True , 'mean':-9.875602358282888e-18, 'std': 63.05380081813939},
                    'transform_te': {'transform': False , 'mean':-9.875602358282888e-18, 'std': 63.05380081813939},
                    }

train_data = ActiveDataHandler(X_train[idxs_train],y_train[idxs_train],transform=transforms_param['transform_tr'])
test_data = ActiveDataHandler(X_test,y_test,transform=transforms_param['transform_te'])
train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=128,
                                        shuffle=True,
                                        collate_fn=collate_fn,
                                        drop_last=True, )
val_dataloader = torch.utils.data.DataLoader(
    test_data,
    batch_size=128,
    collate_fn=collate_fn,
    shuffle=False,
    drop_last=True,
)
trainer.fit(model, train_dataloader, val_dataloader)

C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Users\Patrick Xu\Desktop\RNS_Annotation_Project\active_checkpoints_ent exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params
----------------------------------------
0 | backbone | Sequential | 21.3 M
1 | fc1      | Linear     | 131 K 
2 | fc2      | Linear     | 16.4 K
3 | fc3      | Linear     | 520   
4 | fc4      | Linear     | 18    
5 | softmax  | Softmax    | 0     
----------------------------------------
21.4 M    Trainable params
0         Non-trainable params
21.4 M    Total params
85.732    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\trainer.py:1892: PossibleUserWarning: Th

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [14]:
def random_query(idxs_lb, n):
    inds = np.where(idxs_lb==0)[0]
    return inds[np.random.permutation(len(inds))][:n]

def entropy_query(X_train, y_train, trainer, model, idxs_lb, n):
    idxs_unlabeled = np.arange(n_pool)[~idxs_lb]
    untrained_data = ActiveDataHandler(X_train[idxs_unlabeled], y_train[idxs_unlabeled], transform=transforms_param['transform_te'])
    untrained_dataloader = torch.utils.data.DataLoader(untrained_data,
                                                   batch_size=128,
                                                   shuffle=True,
                                                   collate_fn=collate_fn,
                                                   drop_last=True, )
    predictions = trainer.predict(model,untrained_dataloader)

    probs = []
    m = nn.Softmax(dim=1)
    for pred, y in predictions:
        out = m(pred)
        probs.append(out)
    probs = torch.vstack(probs)
    # print(probs)
    log_probs = torch.log(probs)
    # print(log_probs)
    U = (probs*log_probs).sum(1)
    # print(U.sort())
    # print(U.sort()[1][:n])
    # print(y_train[idxs_unlabeled[U.sort()[1][:n]][::-1]])
    return idxs_unlabeled[U.sort()[1][:n]]

def lease_conf_query(X_train, y_train, trainer, model, idxs_lb, n):
    idxs_unlabeled = np.arange(n_pool)[~idxs_lb]
    untrained_data = ActiveDataHandler(X_train[idxs_unlabeled], y_train[idxs_unlabeled], transform=transforms_param['transform_te'])
    untrained_dataloader = torch.utils.data.DataLoader(untrained_data,
                                                   batch_size=128,
                                                   shuffle=True,
                                                   collate_fn=collate_fn,
                                                   drop_last=True, )
    predictions = trainer.predict(model,untrained_dataloader)
    output_list = []
    m = nn.Softmax(dim=1)
    for pred, y in predictions:
        out = m(pred)
        output_list.append(out)
    probs = torch.vstack(output_list)
    # print(probs)
    U = probs.max(1)[0]
    # print(U)
    # print(y_train[idxs_unlabeled[U.sort()[1][:n]][::-1]])
    return idxs_unlabeled[U.sort()[1][:n]]

In [15]:
for rd in range(1, NUM_ROUND + 1):
    print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
    labeled = len(np.arange(n_pool)[idxs_lb])
    if NUM_QUERY > int(nEnd * n_pool / 100) - labeled:
        NUM_QUERY = int(nEnd * n_pool / 100) - labeled

    output = entropy_query(X_train, y_train, trainer, model, idxs_lb, NUM_QUERY)

    idxs_lb_previous = deepcopy(idxs_lb)
    # output = random_query(idxs_lb, NUM_QUERY)
    q_idxs = output
    idxs_lb_previous[q_idxs] = True
    idxs_lb = idxs_lb_previous
    print(len(np.arange(n_pool)[idxs_lb]))

    idxs_train = np.arange(n_pool)[idxs_lb]
    train_data = ActiveDataHandler(X_train[idxs_train], y_train[idxs_train], transform=transforms_param['transform_tr'])
    test_data = ActiveDataHandler(X_test, y_test, transform=transforms_param['transform_te'])
    train_dataloader = torch.utils.data.DataLoader(train_data,
                                                   batch_size=128,
                                                   shuffle=True,
                                                   collate_fn=collate_fn,
                                                   drop_last=True, )
    val_dataloader = torch.utils.data.DataLoader(
        test_data,
        batch_size=128,
        collate_fn=collate_fn,
        shuffle=False,
        drop_last=True,
    )
    model.load_state_dict(modelstate)
    checkpoint_callback = pl_callbacks.ModelCheckpoint(monitor='val_acc', filename='active_ent_round_' + str(
        rd) + '-{epoch:02d}-{val_loss:.5f}', dirpath='active_checkpoints_ent')
    csv_logger = pl_loggers.CSVLogger("active_logs_ent", name="logger_round_" + str(rd))
    trainer = pl.Trainer(logger=csv_logger, max_epochs=50, callbacks=[checkpoint_callback], accelerator='gpu', devices=1,log_every_n_steps=10)
    trainer.fit(model, train_dataloader, val_dataloader)

Round 1/14


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:489: PossibleUserWarning: Your `predict_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 1it [00:00, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Users\Patrick Xu\Desktop\RNS_Annotation_Project\active_checkpoints_ent exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params
----------------------------------------
0 | backbone | Sequential | 21.3 M
1 | fc1      | Linear     | 131 K 
2 | fc2      | Linear     | 16.4 K
3 | fc3      | Linear     | 520   
4 | fc4      | Linear     | 18    
5 | softmax  | Softmax    | 0     
----------------------------------------
21.4 M    Trainable params
0         Non-trainable params
21.4 M    Total params
85.732    Total estimated model params size (MB)


255


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
